In [1]:
import json
import os
import sys
import transformers 
import torch
import torch.nn.functional as F
import dynalab
from dynalab.handler.base_handler import BaseDynaHandler, ROOTPATH
from dynalab.tasks.hs import TaskIO
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split




In [63]:
>>> from oci.config import from_file
# >>> config = from_file()
??oci.config
# # Using a different profile from the default location
# >>> config = from_file(profile_name="integ-beta")

# # Using the default profile from a different file
# >>> config = from_file(file_location="~/.oci/config.prod")

Object `oci.config` not found.


In [2]:
# NOTE: use the following line to import modules from your repo
sys.path.append(ROOTPATH)
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer
)



In [3]:
# ??AutoModelForSequenceClassification.from_pretrained

In [4]:
# config = AutoConfig.from_pretrained('.')
model = AutoModelForSequenceClassification.from_pretrained('.')
tokenizer = AutoTokenizer.from_pretrained('.')

model.to("cpu")
# model.eval()
# model

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

In [114]:
# model.state_dict()

In [5]:
??DebertaTokenizer


Object `DebertaTokenizer` not found.


In [6]:
# read in data
df = pd.read_csv("data/training_topic_modv2.csv", error_bad_lines=False)
# df.shape
# sample of data 
# sample_df = df.sample(n=5000)
# sample_df.head()
df

,tweet_id,state,cleaned_tweet,positive,neutral,negative,racist,social_justice,food,exclude
0,1.010000e+18,ny,we gotta get back in the lab niggas done stepp...,NaN,1,NaN,0,NaN,NaN,NaN
1,9.890000e+17,tx,just another example about how black womens pa...,NaN,NaN,NaN,0,1,NaN,NaN
2,9.710000e+17,mi,cause be killin these niggas out here,NaN,NaN,1.0,0,NaN,NaN,NaN
3,1.020000e+18,ut,wheres our candy nigga in hawaii lmfao aint bu...,NaN,1,NaN,0,NaN,NaN,NaN
4,1.010000e+18,ga,nah real niggas woke,NaN,1,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9795,9.180000e+17,NaN,nigga paying year in child care not factoring ...,NaN,1,NaN,0,NaN,NaN,NaN
9796,8.850000e+17,tx,ion know if you pull up there my nigga,NaN,1,NaN,0,NaN,NaN,NaN
9797,8.900000e+17,tx,follow the leader ass swag biting ass niggas,NaN,NaN,1.0,0,NaN,NaN,NaN
9798,8.920000e+17,fl,anyone wonder if it about the proposed sale of...,NaN,1,NaN,0,NaN,NaN,NaN


In [7]:
sample_df = df[['cleaned_tweet', 'racist']]
sample_df['racist']= sample_df['racist'].fillna(0)
# s_df = sample_df.sample(n=1000)
sample_df.shape

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(9800, 2)

In [51]:
# hate_df = sample_df[sample_df['racist']==1][:100]

In [52]:
# print(hate_df['cleaned_tweet'])

In [53]:
# no_hate_df = sample_df[sample_df['racist']==0][:100]

In [54]:
# len(hate_df), len(no_hate_df)

(100, 100)

In [8]:
# merged_df = pd.concat([hate_df,no_hate_df])
# merged_df

In [9]:
train_size = 0.8
train_dataset=sample_df.sample(frac=train_size,random_state=100).reset_index(drop=True)
test_dataset=sample_df.drop(train_dataset.index).reset_index(drop=True)

In [10]:
print("FULL Dataset: {}".format(sample_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (9800, 2)
TRAIN Dataset: (7840, 2)
TEST Dataset: (1960, 2)


In [73]:
torch.__version__

'1.5.0'

In [22]:
raicis_df = (train_dataset[train_dataset['racist']==0].count())
raicis_df

cleaned_tweet    6993
racist           6993
dtype: int64

# Re-training the Model

### Get Data

In [26]:
all_data = sample_df;

In [27]:
texts = sample_df['cleaned_tweet'].values
annotations = sample_df['racist'].astype(int).values

### Tokenize & Pipeline

In [28]:
batch_dummy = [tokenizer(i) for i in texts]

In [29]:
# batch_dummy

In [30]:
# lengths of tokens
a = [len(i.input_ids) for i in batch_dummy]
min(a), max(a)

(3, 81)

In [31]:
# no. of words in tweets
b = [len(i.split()) for i in texts]
min(b), max(b)

(1, 60)

In [32]:
batch = [tokenizer(i, padding='max_length', truncation=True, max_length=max(a)) for i in texts]
# batch

In [33]:
min([len(i.input_ids) for i in batch])

81

In [34]:
lbl = []
for l in annotations:
    if l==1:
        lbl.append([1,0])
    else: 
        lbl.append([0,1])
    
        

In [35]:
# set(set(labels))

In [36]:
# # 
labels = torch.tensor(lbl)
# labels

# add logic if 1 => racist, 0 = non racist 

In [37]:
mask = torch.tensor([x.attention_mask for x in batch])

In [38]:
# # make copy of labels tensor, this will be input_ids
# input_ids = labels.detach().clone()


# # create random array of floats with equal dims to input_ids
# rand = torch.rand(input_ids.shape)

# # mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
# mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)

# # loop through each row in input_ids tensor (cannot do in parallel)
# for i in range(input_ids.shape[0]):
#     # get indices of mask positions from mask array
#     selection = torch.flatten(mask_arr[i].nonzero()).tolist()
#     # mask input_ids
#     input_ids[i, selection] = 3  # our custom [MASK] token == 3

### DataLoader

In [39]:
input_ids = []
for i in batch:
    input_i = i.input_ids
    input_ids.append(input_i)
    

In [40]:
print(len(input_ids))

9800


In [41]:
encodings = {'input_ids': input_ids , 'attention_masks': mask, 'labels': labels}

In [42]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
#         for i in batch:
#             input_ids = batch['input_ids']
        return torch.tensor(self.encodings['input_ids']) .shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: torch.tensor(tensor[i],dtype=torch.int64) for key, tensor in self.encodings.items()}

In [43]:
dataset = Dataset(encodings)

In [44]:
loader = torch.utils.data.DataLoader(dataset, batch_size=len(input_ids), shuffle=True)

In [45]:
len(input_ids)

9800

### Training

In [46]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification, DebertaConfig
from transformers import AdamW
from tqdm.auto import tqdm

In [47]:
config = DebertaConfig()
#     vocab_size=50265,
#     hidden_size=768,
#     num_hidden_layers=12,
#     num_attention_heads=12,
#     intermediate_size=3072,
#     hidden_act='gelu',
#     hidden_dropout_prob=0.1,
#     attention_probs_dropout_prob=0.1,
#     max_position_embeddings=512,
#     type_vocab_size=0,
#     initializer_range=0.02,
#     layer_norm_eps=1e-07,
#     relative_attention=False,
#     max_relative_positions=-1,
#     pad_token_id=0,
#     position_biased_input=True,
#     pos_att_type=None,
#     pooler_dropout=0,
#     pooler_hidden_act='gelu',
#     **kwargs,

In [48]:
# model = DebertaForSequenceClassification(config)


In [53]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
# model.to(device)

In [50]:
# with open('pytorch_model.bin', mode='rb') as file:
#     fileContent = file.read()
#     print(fileContent)

In [54]:
model.train(mode=True)
optim = AdamW(model.parameters(), lr=1e-4)

In [55]:
epochs = 11

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_masks'].to(device)
        labels = labels.to(device)
#         print(input_ids[0])
        # process
        outputs = model(input_ids,attention_mask=attention_mask,
                        labels=labels) 
        
#  
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/1 [00:00<?, ?it/s]

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


KeyboardInterrupt: 

In [41]:
len(outputs.logits)

20

In [119]:
from sklearn.metrics import precision_recall_fscore_support


In [120]:
test = merged_df.sample(n=100)

In [195]:
test_texts = test['cleaned_tweet']
test_texts

612     walk them dogs if islamist dont like it go bac...
1381                                       ima deport you
58         nigga why you tripping get your mood right yah
1388    this lady is horribly racist busy body who sho...
961     man not even thirty seconds after posted that ...
                              ...                        
26              swear kyra this niggas lips were burgundy
130     go lecture to ur fellow jihadis stop wasting m...
47          anything telling nigga that hes grown ass man
1447    look these immigrants are either bad or me nee...
1102    absolutely set up go fund the wall and start d...
Name: cleaned_tweet, Length: 100, dtype: object

In [196]:
# tokenize and output inference 
all_inference_output_tweets= []
for tweet in test_texts:
    input_tweets = tokenizer(tweet, return_tensors="pt")
    with torch.no_grad():
# #     print(input_tweets)
        inference_output_tweets = model(**input_tweets)
# #     print(inference_output_tweets)
        inference_output_tweets = F.softmax(inference_output_tweets[0]).squeeze()
        all_inference_output_tweets.append(inference_output_tweets)
    

#         print(inference_output_tweets)

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [141]:
print(inference_output_tweets)

tensor([0.0838, 0.9162])


In [197]:
# ID and Model Labeling 
labeled_tweets = []
for i in range(len(test)):
    response= {}
    index = int(all_inference_output_tweets[i].argmax())
    response["label"] = {0: "0",1: "1"}[index]
    response['prob'] = all_inference_output_tweets[i][index]
    response['id']= test['cleaned_tweet'].iloc[i]
    labeled_tweets.append(response)
new_df = pd.DataFrame(labeled_tweets)

response["prob"] = {"not-hateful": float(inference_output_tweets[0]), "hateful": float(inference_output_tweets[1])}

In [198]:
new_df

,label,prob,id
0,1,tensor(0.9921),walk them dogs if islamist dont like it go bac...
1,0,tensor(0.8940),ima deport you
2,1,tensor(0.9009),nigga why you tripping get your mood right yah
3,0,tensor(0.7799),this lady is horribly racist busy body who sho...
4,1,tensor(0.7349),man not even thirty seconds after posted that ...
...,...,...,...
95,1,tensor(0.9914),swear kyra this niggas lips were burgundy
96,1,tensor(0.8858),go lecture to ur fellow jihadis stop wasting m...
97,1,tensor(0.9639),anything telling nigga that hes grown ass man
98,1,tensor(0.9760),look these immigrants are either bad or me nee...


In [199]:
new_df = new_df.rename(columns={"id": "cleaned_tweet"})
new_df.shape

(100, 3)

In [200]:
merged_df = pd.merge(new_df,test, on = "cleaned_tweet", how = "outer")
merged_df.tail()                    

,label,prob,cleaned_tweet,racist
95,1,tensor(0.9914),swear kyra this niggas lips were burgundy,0
96,1,tensor(0.8858),go lecture to ur fellow jihadis stop wasting m...,1
97,1,tensor(0.9639),anything telling nigga that hes grown ass man,0
98,1,tensor(0.9760),look these immigrants are either bad or me nee...,1
99,1,tensor(0.9162),absolutely set up go fund the wall and start d...,1


In [201]:
annts = (test['racist'])
annts


612     1
1381    1
58      0
1388    1
961     1
       ..
26      0
130     1
47      0
1447    1
1102    1
Name: racist, Length: 100, dtype: int64

In [202]:
labeled_tweets

[{'label': '1',
  'prob': tensor(0.9921),
  'id': 'walk them dogs if islamist dont like it go back where you came from'},
 {'label': '0', 'prob': tensor(0.8940), 'id': 'ima deport you'},
 {'label': '1',
  'prob': tensor(0.9009),
  'id': 'nigga why you tripping get your mood right yah'},
 {'label': '0',
  'prob': tensor(0.7799),
  'id': 'this lady is horribly racist busy body who should be heavily fined for not only wasting the policemens time but harassing these poor people just trying to enjoy the nice weather id bet if they were white there would have been no call'},
 {'label': '1',
  'prob': tensor(0.7349),
  'id': 'man not even thirty seconds after posted that saw like four pics of her with no makeup and she defly look like trailer trash who ran into some money and got booty'},
 {'label': '0',
  'prob': tensor(0.6528),
  'id': 'specifically called out two congress people both of whom happen to be women and african americans as illiterate ive asked her repeatedly for examples of the

In [203]:
y_pred  = [a_dict['label'] for a_dict in labeled_tweets]
y_pred = np.array(y_pred, dtype=np.int64)

In [204]:
annts = np.array(annts, dtype=np.int64)

In [205]:
precision_recall_fscore_support(annts, y_pred, average='macro')
precision_recall_fscore_support(annts, y_pred, average='micro')
precision_recall_fscore_support(annts, y_pred, average='weighted')


(0.49559845559845556, 0.49, 0.48148193136957174, None)

# Evaluate Model

In [117]:
new= "I love heran"

In [111]:
input_tweets = tokenizer(new, return_tensors="pt")

In [112]:
input_tweets

{'input_ids': tensor([[   1,  282, 1610, 3418,  154,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [113]:
tweets_output = model(**input_tweets)

In [114]:
tweets_output

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.8463, -3.3395]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [116]:
response = {}
index = int(inference_output_tweets.argmax())
response["label"] = {0: "not-hateful",1: "hateful"}[index]
response['prob'] = inference_output_tweets[index]
response['id']= new
# labeled_tweets.append (response)
# new_df = pd.DataFrame(response, index=1)
response

{'label': 'not-hateful',
 'prob': tensor(0.9979, grad_fn=<SelectBackward>),
 'id': 'nbe wilding'}

### Evaluate

#  Pre-Trained Model

In [82]:
# # tokenize and output inference 
# all_inference_output_tweets= []
# for tweet in texts:
#     input_tweets = tokenizer(tweet, return_tensors="pt")
#     with torch.no_grad():
# # #     print(input_tweets)
#         inference_output_tweets = model(**input_tweets)
# # #     print(inference_output_tweets)
#         inference_output_tweets = F.softmax(inference_output_tweets[0]).squeeze()
#         all_inference_output_tweets.append(inference_output_tweets)
    

# #         print(inference_output_tweets)
    

In [83]:
# # ID and Model Labeling 
# labeled_tweets = []
# for i in range(len(merged_df)):
#     response= {}
#     index = int(all_inference_output_tweets[i].argmax())
#     response["label"] = {0: "not-hateful",1: "hateful"}[index]
#     response['prob'] = all_inference_output_tweets[i][index]
#     response['id']= sample_df['cleaned_tweet'].iloc[i]
#     labeled_tweets.append(response)
# new_df = pd.DataFrame(labeled_tweets)

# response["prob"] = {"not-hateful": float(inference_output_tweets[0]), "hateful": float(inference_output_tweets[1])}

In [81]:
new_df['label'].unique()

array(['hateful'], dtype=object)

In [ ]:
new_df = new_df.rename(columns={"id": "cleaned_tweet"})
new_df.shape

In [ ]:
# sample_df

In [ ]:
# merged_df = pd.merge(new_df,sample_df, on = "cleaned_tweet", how = "outer")
# merged_df.tail()                    

In [ ]:
# matched_hateful = merged_df[(merged_df['racist']==1) & (merged_df['label']=='hateful')]
# print(len(matched_hateful))

In [ ]:
# umatched_hateful = merged_df[(merged_df['racist']==0) & (merged_df['label']=='hateful')]
# print(len(umatched_hateful))

In [ ]:
# matched_not_hateful = merged_df[(merged_df['racist']==0) & (merged_df['label']=='not-hateful')]
# print(len(matched_not_hateful))

In [ ]:
# umatched_not_hateful = merged_df[(merged_df['racist']==1) & (merged_df['label']=='not-hateful')]
# print(len(umatched_not_hateful))

In [ ]:
# not_hateful = merged_df[(merged_df['hate_speech']==1) & (merged_df['label']=='not-hateful')]
# text = not_hateful['tweet']
# text.values

In [ ]:
# merged_df.to_csv('merged.csv')

In [ ]:
# import simpletransformers
# from simpletransformers.classification import (
#     ClassificationModel, ClassificationArgs
# )
# import pandas as pd
# import logging

In [ ]:



# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Preparing train data
# train_data = [
#     [
#         " I hate black people",
#         "Send Asians Back",
#         1,
#     ],
#     [
#         "I do not feel like working",
#         "Love Ethiopian Food",
#         0,
#     ],
# ]
# train_df = pd.DataFrame(train_data)
# train_df.columns = ["text_a", "text_b", "labels"]

# # Preparing eval data
# eval_data = [
#     [
#         "Black Lives Do not matter ",
#         "Build the wall",
#         1,
#     ],
#     [
#         "Merry was the king of Rohan",
#         "Legolas was taller than Gimli",
#         0,
#     ],
# ]
# eval_df = pd.DataFrame(eval_data)
# eval_df.columns = ["text_a", "text_b", "labels"]

# # Optional model configuration
# model_args = ClassificationArgs(num_train_epochs=1)

# # Create a ClassificationModel
# model = ClassificationModel("deberta", "microsoft/deberta-base", use_cuda=False)

# # Train the model
# model.train_model(train_df, output_dir='outputs' )

# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(
#     eval_df
# )

# # Make predictions with the model
# predictions, raw_outputs = model.predict(
#     [
#         [
#             "Fuck covid asians",
#             "Muslims should not come",
#         ]
#     ]
# )

# # from models.multi_choice import MultiChoiceModel

In [ ]:
# print(predictions)